In [46]:
import pandas as pd
import numpy as np
import warnings
import pickle
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
from scipy.special import expit as sigmoid
import sys

import torch
from torch import nn
from torch.optim import Adam
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import os


In [47]:

#-----------------------------------------------------------------------#
def loop_body(t_true, t_pred, i, ta):
    '''
    This function is the main body of the custom_loss() definition, called from within the torch.while_loop()
    The loss function implemented here is as described in the original YOLO paper: https://arxiv.org/abs/1506.02640

    # Arguments
    t_true: the ground truth tensor; shape: (batch_size, 1573)
    t_pred: the predicted tensor; shape: (batch_size, 1573)
    i: iteration count of the while_loop
    ta: List to store loss tensors
    '''

    ### Get the current iteration's true and predicted tensor
    c_true = t_true[i]
    c_pred = t_pred[i]

    ### Apply sigmoid to the coordinates part of the tensor to scale it between 0 and 1 as expected
    c_pred = torch.cat((c_pred[:605], torch.sigmoid(c_pred[-968:])), dim=0)


    ### Reshape to GRIDxGRIDxBBOXES blocks for simpler correspondence of values across grid cell and bounding boxes
    xywh_true = c_true[-968:].view(11, 11, 2, 4)
    xywh_pred = c_pred[-968:].view(11, 11, 2, 4)

    ### Convert normalized values to actual ones (still relative to grid cell size)
    X_NORM = 1.0  # Replace with the actual normalization value for 'X'
    Y_NORM = 1.0  # Replace with the actual normalization value for 'Y'
    WIDTH_NORM = 1.0  # Replace with the actual normalization value for 'Width'
    HEIGHT_NORM = 1.0  # Replace with the actual normalization value for 'Height'

    GRID_NUM = 11
    X_SPAN = WIDTH_NORM/GRID_NUM
    Y_SPAN = HEIGHT_NORM/GRID_NUM
    X_NORM = WIDTH_NORM/GRID_NUM
    Y_NORM = HEIGHT_NORM/GRID_NUM
    WIDTH_NORM = 224
    HEIGHT_NORM = 224



    x_true = xywh_true[:, :, :, 0] * X_NORM
    x_pred = xywh_pred[:, :, :, 0] * X_NORM

    y_true = xywh_true[:, :, :, 1] * Y_NORM
    y_pred = xywh_pred[:, :, :, 1] * Y_NORM

    w_true = xywh_true[:, :, :, 2] * WIDTH_NORM
    w_pred = xywh_pred[:, :, :, 2] * WIDTH_NORM

    h_true = xywh_true[:, :, :, 3] * HEIGHT_NORM
    h_pred = xywh_pred[:, :, :, 3] * HEIGHT_NORM

    # Remaining code to be converted similarly...

    # Example: For tensor arithmetic operations, torch.abs(), torch.sqrt(), torch.square(), etc.
    # Also, use torch.where() instead of tf.nn.relu() for the ReLU operation.

    # Remaining code that follows tensor operations should be similarly converted.

    # For PyTorch, make sure to use appropriate tensor operations to perform the same calculations.
    # Ensure to replace tensor slicing and concatenation with PyTorch equivalents.

    # ...

    ### The below is a different approach on calculating IOU between
    ### predicted bounding boxes and ground truth
    ### See README.md for explanation for the formula

    x_dist = torch.abs(torch.subtract(x_true, x_pred))
    y_dist = torch.abs(torch.subtract(y_true, y_pred))


    ### (w1/2 +w2/2 -d) > 0 => intersection, else no intersection
    ### (h1/2 +h2/2 -d) > 0 => intersection, else no intersection

    wwd = torch.nn.functional.relu(w_true/2 + w_pred/2 - x_dist)
    hhd = torch.nn.functional.relu(h_true/2 + h_pred/2 - y_dist)

    area_true = torch.multiply(w_true, h_true)
    area_pred = torch.multiply(w_pred, h_pred)
    area_intersection = torch.multiply(wwd, hhd)

    iou = area_intersection / (area_true + area_pred - area_intersection + 1e-4)
    confidence_true = torch.reshape(iou, (-1,))

    ### Masks for grids that do contain an object, from ground truth
    ### The class probability block from the ground truth is used as an indicator for all grid cells that
    ### actually have an object present in itself.
    grid_true = torch.reshape(c_true[:363], (11,11,3))
    grid_true_sum = torch.sum(grid_true, dim=2)
    grid_true_exp = torch.stack((grid_true_sum, grid_true_sum), dim=2)
    grid_true_exp3 = torch.stack((grid_true_sum, grid_true_sum, grid_true_sum), dim=2)
    grid_true_exp4 = torch.stack((grid_true_sum, grid_true_sum, grid_true_sum, grid_true_sum), dim=2)

    coord_mask = torch.reshape(grid_true_exp4, (-1,))
    confidence_mask = torch.reshape(grid_true_exp, (-1,))
    confidence_true = confidence_true * confidence_mask

    ### Revised ground truth tensor, based on calculated confidence values and with non-object grids suppressed
    c_true_new = torch.cat([c_true[:363], confidence_true, c_true[-968:]], dim=0)

    ### Create masks for 'responsible' bounding box in a grid cell for loss calculation
    confidence_true_matrix = torch.reshape(confidence_true, (11,11,2))
    confidence_true_argmax = torch.argmax(confidence_true_matrix, dim=2)
    confidence_true_argmax = confidence_true_argmax.type(torch.int32)
    ind_i, ind_j = torch.meshgrid(torch.arange(11, dtype=torch.int32), torch.arange(11, dtype=torch.int32), indexing='ij')
    ind_argmax = torch.stack((ind_i, ind_j, confidence_true_argmax), axis=2)
    ind_argmax = torch.reshape(ind_argmax, (121,3))

    ind_argmax = ind_argmax.type(torch.int64)


    # responsible_mask_2 = torch.full((11,11,2), 0).scatter_(1, ind_argmax,1)
    # responsible_mask_2 = torch.reshape(responsible_mask_2, (-1,))
    # responsible_mask_2 = responsible_mask_2 * confidence_mask

    # responsible_mask_4 = torch.full((11,11,2,2), 0).scatter_(1, ind_argmax,1)
    # responsible_mask_4 = torch.reshape(responsible_mask_4, (-1,))
    # responsible_mask_4 = responsible_mask_4 * coord_mask

    responsible_mask_2 = torch.zeros(11, 11, 2)
    responsible_mask_2[ind_argmax[:, 0], ind_argmax[:, 1], ind_argmax[:, 2]] = 1
    responsible_mask_2 = responsible_mask_2.view(-1)
    responsible_mask_2 = responsible_mask_2 * confidence_mask

    responsible_mask_4 = torch.zeros(11, 11, 2, 2)
    responsible_mask_4[ind_argmax[:, 0], ind_argmax[:, 1], ind_argmax[:, 2], :] = 1
    responsible_mask_4 = responsible_mask_4.view(-1)
    responsible_mask_4 = responsible_mask_4 * coord_mask


    ### Masks for rest of the bounding boxes
    responsible_mask_2.type(torch.bool)
    responsible_mask_4.type(torch.bool)

    negated_responsible_mask_2 = torch.logical_not(responsible_mask_2)
    negated_responsible_mask_4 = torch.logical_not(responsible_mask_4)
    inv_responsible_mask_2 = negated_responsible_mask_2.type(torch.float32)
    inv_responsible_mask_4 = negated_responsible_mask_4.type(torch.float32)

    ### lambda values
    lambda_coord = 5.0
    lambda_noobj = 0.5


    ### loss from dimensions ###
    dims_true = torch.reshape(c_true_new[-968:], (11,11,2,4))
    dims_pred = torch.reshape(c_pred[-968:], (11,11,2,4))

    xy_true = torch.reshape(dims_true[:,:,:,:2], (-1,))
    xy_pred = torch.reshape(dims_pred[:,:,:,:2], (-1,))

    wh_true = torch.reshape(dims_true[:,:,:,2:], (-1,))
    wh_pred = torch.reshape(dims_pred[:,:,:,2:], (-1,))


    #### XY difference loss
    xy_loss = (xy_true - xy_pred) * responsible_mask_4
    xy_loss = torch.square(xy_loss)
    xy_loss = lambda_coord * torch.sum(xy_loss)


    #### WH sqrt diff loss
    wh_loss = (torch.sqrt(torch.abs(wh_true)) - torch.sqrt(torch.abs(wh_pred))) * responsible_mask_4
    wh_loss = torch.square(wh_loss)
    wh_loss = lambda_coord * torch.sum(wh_loss)


    ### Conf losses
    conf_true = c_true_new[363:605]
    conf_pred = c_pred[363:605]


    conf_loss_obj = (conf_true - conf_pred) * responsible_mask_2
    conf_loss_obj = torch.square(conf_loss_obj)
    conf_loss_obj = torch.sum(conf_loss_obj)


    conf_loss_noobj = (conf_true - conf_pred) * inv_responsible_mask_2
    conf_loss_noobj = torch.square(conf_loss_noobj)
    conf_loss_noobj = lambda_noobj * torch.sum(conf_loss_noobj)


    #### Class Prediction Loss
    class_true = torch.reshape(c_true_new[:363], (11,11,3))
    class_pred = torch.reshape(c_pred[:363], (11,11,3))
    class_pred_softmax = class_pred #tf.nn.softmax(class_pred)

    classification_loss = class_true - class_pred_softmax
    classification_loss = classification_loss * grid_true_exp3
    classification_loss = torch.square(classification_loss)
    classification_loss = torch.sum(classification_loss)

    ## Total loss = xy-loss + wh-loss + Confidence_loss_obj + Confidence_loss_noobj + classification_loss
    total_loss = xy_loss + wh_loss + conf_loss_obj + conf_loss_noobj + classification_loss



    total_loss.requires_grad_(True)

    # Finally, append the calculated loss to the list ta
    ta = total_loss.clone().detach().requires_grad_(True)  # Replace this with the calculated loss



    i = i + 1
    return t_true, t_pred, i, ta

def custom_loss(y_true, y_pred):
    '''
    Custom loss function as per the YOLO paper, since there are no default loss functions in PyTorch that fit.
    '''
    i = 0
    ta = []
    y_true = y_true.reshape(-1,1573)
    while i < y_true.size(0):  # Assuming y_true is a torch tensor
        y_true, y_pred, i, ta_i_value = loop_body(y_true, y_pred, i, ta)
        ta.append(ta_i_value)

    # Once the loop is done, calculate mean loss from the list of losses
    loss_tensor = torch.tensor(ta)
    loss_mean = torch.mean(loss_tensor)

    if not isinstance(loss_mean, torch.Tensor):
        # print("not a tensor")
        loss_mean = torch.tensor(loss_mean, requires_grad=True)
    elif not loss_mean.requires_grad:
        # print("is a tensor")
        loss_mean.requires_grad_(True)


    return loss_mean  # Return the mean loss

#-----------------------------------------------------------------------#

### Helper funtions for data augumentation for training the network ###
def coord_translate(bboxes, tr_x, tr_y):
    '''
    Takes a single frame's bounding box list with confidence scores and
    applies translation (addition) to the coordinates specified by 'tr'

    parameters:
    bboxes: list with element of the form ((x1,y1), (x2,y2)), (c1,c2,c3)
    tr_x, tr_y: translation factor to add the coordinates to, for x and y respectively

    returns: new list with translated coordinates and same conf scores; same shape as bboxes
    '''
    new_list = []
    for box in bboxes:
        coords = np.array(box[0])
        coords[:,0] = coords[:,0] + tr_x
        coords[:,1] = coords[:,1] + tr_y
        coords = coords.astype(np.int64)
        out_of_bound_indices = np.average(coords, axis=0) >= 224
        if out_of_bound_indices.any():
            continue
        coords = coords.tolist()
        new_list.append((coords, box[1]))
    return new_list
def coord_scale(bboxes, sc):
    '''
    Takes a singl frame's bounding box list with confidence scores and
    applies scaling to the coordinates specified by sc

    parameters:
    bboxes: list with element of the form ((x1,y1), (x2,y2)), (c1,c2,c3)
    sc: scaling factor to multiply the coordinates with

    returns: new list with scaled coordinates and same conf scores; same shape as bboxes
    '''
    new_list = []
    for box in bboxes:
        coords = np.array(box[0])
        coords = coords * sc
        coords = coords.astype(np.int64)
        out_of_bound_indices = np.average(coords, axis=0) >= 224
        if out_of_bound_indices.any():
            continue
        coords = coords.tolist()
        new_list.append((coords, box[1]))
    return new_list
def label_to_tensor(frame, imgsize=(224, 224), gridsize=(11,11), classes=3, bboxes=2):

    '''
    This function takes in the frame (rows corresponding to a single image in the labels.csv)
    and converts it into the format our network expects (coord conversion and normalization)

    '''
    grid = np.zeros(gridsize)

    y_span = imgsize[0]/gridsize[0]
    x_span = imgsize[1]/gridsize[1]

    class_prob = np.zeros((gridsize[0], gridsize[1], classes))
    confidence = np.zeros((gridsize[0], gridsize[1], bboxes))
    dims = np.zeros((gridsize[0], gridsize[1], bboxes, 4))

    for box in frame:
        ((x1,y1), (x2,y2)), (c1,c2,c3) = box
        x_grid = int(((x1+x2)/2)//x_span)
        y_grid = int(((y1+y2)/2)//y_span)

        class_prob[y_grid, x_grid] = (c1,c2,c3)

        x_center = ((x1+x2)/2)
        y_center = ((y1+y2)/2)

        x_center_norm = (x_center-x_grid*x_span)/(x_span)
        y_center_norm = (y_center-y_grid*y_span)/(y_span)

        w = x2-x1
        h = y2-y1

        w_norm = w/imgsize[1]
        h_norm = h/imgsize[0]

        dims[y_grid, x_grid, :, :] = (x_center_norm, y_center_norm, w_norm, h_norm)

        grid[y_grid, x_grid] += 1

    tensor = np.concatenate((class_prob.ravel(), confidence.ravel(), dims.ravel()))
    return tensor

# TODO change the "folder" argument to where the images are residing
def augument_data(label, frame, imgsize=(224, 224), folder=r'D:/PD Data/Resnet+yolo Code/Plant Disease CSV Format 3 Class more Train Data/train/'):
    '''
    Takes the image file name and the frame (rows corresponding to a single image in the labels.csv)
    and randomly scales, translates, adjusts SV values in HSV space for the image,
    and adjusts the coordinates in the 'frame' accordingly, to match bounding boxes in the new image
    '''
    img = cv2.imread(folder+label)
    img = cv2.resize(img, imgsize)
    rows, cols = img.shape[:2]

    #translate_factor
    tr = np.random.random() * 0.2 + 0.01
    tr_y = np.random.randint(rows*-tr, rows*tr)
    tr_x = np.random.randint(cols*-tr, cols*tr)
    #scale_factor
    sc = np.random.random() * 0.4 + 0.8

    # flip coin to adjust image saturation
    r = np.random.rand()
    if r < 0.5:
        #randomly adjust the S and V values in HSV representation
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV).astype(np.float32)
        fs = np.random.random() + 0.7
        fv = np.random.random() + 0.2
        img[:,:,1] *= fs
        img[:,:,2] *= fv
        img = img.astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)

    # new random factor for scaling and translating
    r = np.random.rand()

    if r < 0.3:
        #translate image
        M = np.float32([[1,0,tr_x], [0,1,tr_y]])
        img = cv2.warpAffine(img, M, (cols,rows))
        frame = coord_translate(frame, tr_x, tr_y)
    elif r < 0.6:
        #scale image keeping the same size
        placeholder = np.zeros_like(img)
        meta = cv2.resize(img, (0,0), fx=sc, fy=sc)
        if sc < 1:
            placeholder[:meta.shape[0], :meta.shape[1]] = meta
        else:
            placeholder = meta[:placeholder.shape[0], :placeholder.shape[1]]
        img = placeholder
        frame = coord_scale(frame, sc)

    return img, frame
#-----------------------------------------------------------------------#

### Define generator and Import dataset (do test/train split)
# would need to look into this
def generator(label_keys, label_frames, batch_size=1, folder='udacity-object-detection-crowdai/'):
    '''
    Generator function
    # Arguments
    label_keys: image names, that are keys of the label_frames Arguments
    label_frames: array of frames (rows corresponding to a single image in the labels.csv)
    batch_size: batch size
    '''
    num_samples = len(label_keys)
    indx = label_keys

    count = 0
    while count < 1:
        shuffle(indx)
        for offset in range(0, num_samples, batch_size):
            batch_samples = indx[offset:offset+batch_size]

            images = []
            gt = []
            for batch_sample in batch_samples:
                im, frame = augument_data(batch_sample, label_frames[batch_sample])
                im = im.astype(np.float32)
                im -= 128
                images.append(im)
                frame_tensor = label_to_tensor(frame)
                gt.append(frame_tensor)

            X_train = torch.Tensor(images)
            y_train = torch.Tensor(gt)
            yield shuffle(X_train, y_train)

        count += 1

def plot_history(history_object):
    print(history_object.history.keys())
    ### plot the training and validation loss for each epoch
    plt.plot(history_object.history['loss'])
    plt.plot(history_object.history['val_loss'])
    plt.title('model mean squared error loss')
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    plt.legend(['training set', 'validation set'], loc='upper right')
    plt.show()


In [48]:
def iou_value(box1, box2):
    '''
    calculate the IOU of two given boxes
    '''
    (x11, y11) , (x12, y12) = box1
    (x21, y21) , (x22, y22) = box2

    x1 = max(x11, x21)
    x2 = min(x12, x22)
    w = max(0, (x2-x1))

    y1 = max(y11, y21)
    y2 = min(y12, y22)
    h = max(0, (y2-y1))

    area_intersection = w*h
    area_combined = abs((x12-x11)*(y12-y11) + (x22-x21)*(y22-y21) + 1e-3)

    return area_intersection/area_combined


def find_average_iou(bounding_boxes_grouped_actual, bounding_boxes_grouped_predicted):

    all_ious = []
    for bb_actual,bb_predicted in zip(bounding_boxes_grouped_actual, bounding_boxes_grouped_predicted):

        if bb_actual[0].sum() <= 0:
            continue

        x_act,y_act,w_act,h_act = bb_actual[0]
        x_pred,y_pred,w_pred,h_pred = bb_predicted[0]

        bbx_act = ((x_act-w_act/2,y_act-h_act/2), (x_act+w_act/2,y_act+h_act/2))
        bbx_pred = ((x_pred-w_pred/2,y_pred-h_pred/2), (x_pred+w_pred/2,y_pred+h_pred/2))

        iou = iou_value(bbx_act, bbx_pred)

        if iou >0 :
            all_ious.append(iou)

    return all_ious

In [49]:
model = torch.hub.load(
    'AdeelH/pytorch-fpn',
    'make_fpn_resnet',
    force_reload=True,
    name='resnet50',
    fpn_type='panet',
    num_classes=2,
    fpn_channels=256,
    in_channels=3,
    out_size=(224, 224)
)

# for weight in model.parameters():
#     weight.requires_grad = False


class CustomYolo(nn.Module):
    def __init__(self):
        super(CustomYolo,self).__init__()
        self.model = model
        self.flat_layer = nn.Flatten(0,-1)


        # output tensor :
        # SS: Grid cells: 11*11
        # B: Bounding box per grid cell: 2
        # C: classes: 3
        # Coords: x, y, w, h per box: 4
        # tensor length: SS * (C +B(5) ) : 363--242--968 => 1573
        out_dim = 11*11*(3+2*5)
        self.dense_0 = nn.Linear(100352, out_dim)
        # self._fc = nn.Linear(1024,200, bias=False)
    def forward(self, x):
        batch_size ,_,_,_ =x.shape
        x = self.model(x)
        x = self.flat_layer(x)
        x = self.dense_0(x)
        # x = self.model.avgpool(x)
        # x = x.view(-1, 1024)
        # x = self.layernorm(x)
        # x = self._fc(x)
        # x = F.normalize(x, p=2, dim=1)
        return x



Downloading: "https://github.com/AdeelH/pytorch-fpn/zipball/master" to C:\Users\prompt/.cache\torch\hub\master.zip
C:\Users\prompt\anaconda3\envs\PD\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\prompt\anaconda3\envs\PD\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
1e-2

0.01

In [50]:

train_loss_csv = pd.DataFrame(columns=["Loss", "IOU"])
test_loss_csv = pd.DataFrame(columns=["Loss", "IOU"])


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

WIDTH_NORM = 224
HEIGHT_NORM = 224
GRID_NUM = 11
X_SPAN = WIDTH_NORM/GRID_NUM
Y_SPAN = HEIGHT_NORM/GRID_NUM
X_NORM = WIDTH_NORM/GRID_NUM
Y_NORM = HEIGHT_NORM/GRID_NUM

weights_path = 'imagenet'
save_prefix = 'run1_'
learning_rate = 1e-2
if len(sys.argv) > 3:
    weights_path = sys.argv[1]
    save_prefix = sys.argv[2]
    learning_rate = float(sys.argv[3])
elif len(sys.argv) > 2:
    weights_path = sys.argv[1]
    save_prefix = sys.argv[2]
elif len(sys.argv) > 1:
    weights_path = sys.argv[1]


yolo_resnet = CustomYolo()

tfm = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


yolo_resnet = yolo_resnet.to(device)

#TODO Change the file path here if needed
with open(f"D:\PD Data\Resnet+yolo Code\label_frames_plant_disease_final_v_train (1).p", 'rb') as f:
    label_frames = pickle.load(f)

label_keys = list(label_frames.keys())
lbl_train, lbl_validn = train_test_split(label_keys, test_size=0.2)

LEN_TRAIN = len(lbl_train)
LEN_TEST = len(lbl_validn)


### Intialize generator
optimiser = Adam(yolo_resnet.parameters(), lr=1e-2, weight_decay=0.01)


# training_image = torch.from_numpy(train_generator[0])
# training_image = training_image.permute(0, 3, 1, 2)

train_losses = []
test_losses = []
train_iou_list = []
test_iou_list = []


segment_lengths = [363, 242, 968]


for epoch in range(20):
    # start = time()

    tr_acc = 0
    test_acc = 0

    # Train
    yolo_resnet.train()
    # count = 0

    # model_weights_path = f"yolo_resnet_weights_epoch{epoch + 1}.pth"
    # torch.save(yolo_resnet.state_dict(), model_weights_path)
    # print(f"Model weights saved for Epoch {epoch + 1}")

    train_generator = generator(lbl_train, label_frames)

    with tqdm(train_generator, unit="batch") as tepoch:
        train_loss_for_mean = []
        for x_train, y_train in tepoch:
            optimiser.zero_grad()

            y_train_segments = [
                    y_train[0][:segment_lengths[0]],
                    y_train[0][segment_lengths[0]:segment_lengths[0] + segment_lengths[1]],
                    y_train[0][segment_lengths[0] + segment_lengths[1]:]
                ]


            class_probabilities_actual = y_train_segments[0]  # Segment 1
            confidence_actual = y_train_segments[1]  # Segment 2
            bounding_boxes_actual = y_train_segments[2]  # Segment 3

            bounding_boxes_grouped_actual = bounding_boxes_actual.reshape(-1,2,4)
            class_probabilities_grouped_actual = class_probabilities_actual.reshape(-1, 3)
            confidence_grouped_actual = confidence_actual.reshape(-1,2)


            x_train = x_train.permute(0, 3, 1, 2)
            x_train = x_train.to(device)

            train_prob = yolo_resnet(x_train)
            train_prob = train_prob.cpu()

            train_prob_segments = [
                    train_prob[:segment_lengths[0]],
                    train_prob[segment_lengths[0]:segment_lengths[0] + segment_lengths[1]],
                    train_prob[segment_lengths[0] + segment_lengths[1]:]
                ]


            class_probabilities_predicted = train_prob_segments[0]  # Segment 1
            confidence_predicted = train_prob_segments[1]  # Segment 2
            bounding_boxes_predicted = train_prob_segments[2]  # Segment 3

            bounding_boxes_grouped_predicted = bounding_boxes_predicted.reshape(-1,2,4)
            class_probabilities_grouped_predicted = class_probabilities_predicted.reshape(-1, 3)
            confidence_grouped_predicted = confidence_predicted.reshape(-1,2)


            train_loss = custom_loss(train_prob, y_train)
            train_loss.backward()
            optimiser.step()
            train_loss_for_mean.append(train_loss)

            
                        # Inside the loop where you calculate IoU
            if bounding_boxes_grouped_predicted.numel() == 0:
                print("No predicted bounding boxes.")
                average_train_iou = 0
            else:
                iou_list = find_average_iou(bounding_boxes_grouped_actual, bounding_boxes_grouped_predicted)
                average_train_iou = 0 if len(iou_list) == 0 else sum(iou_list) / len(iou_list)
                print(iou_list)
            train_iou_list.append(average_train_iou)

            # iou_list = find_average_iou(bounding_boxes_grouped_actual, bounding_boxes_grouped_predicted)

            # if len(iou_list) == 0:
            #     print("iou_list = 0")
            #     average_train_iou = 0
            # else:
            #     average_train_iou = sum(iou_list)/len(iou_list)
            #     print(iou_list)
            # train_iou_list.append(average_train_iou)




            # training ends
            # train_pred = torch.max(train_prob, 1).indices
            # tr_acc += int(torch.sum(train_pred == y_train))

    average_train_loss = sum(train_loss_for_mean)/len(train_loss_for_mean)
    train_losses.append(average_train_loss)

    # ep_tr_acc = tr_acc / LEN_TRAIN


        # ep_tr_acc = tr_acc / LEN_TRAIN

    # Evaluate
    validation_generator = generator(lbl_validn, label_frames)

    yolo_resnet.eval()
    with torch.no_grad():
        test_loss_for_mean = []
        for xtest, ytest in validation_generator:
            xtest = xtest.to(device)
            xtest = xtest.permute(0, 3, 1, 2)

            y_test_segments = [
                    ytest[0][:segment_lengths[0]],
                    ytest[0][segment_lengths[0]:segment_lengths[0] + segment_lengths[1]],
                    ytest[0][segment_lengths[0] + segment_lengths[1]:]
                ]


            class_probabilities_actual = y_test_segments[0]  # Segment 1
            confidence_actual = y_test_segments[1]  # Segment 2
            bounding_boxes_actual = y_test_segments[2]  # Segment 3

            bounding_boxes_grouped_actual = bounding_boxes_actual.reshape(-1,2,4)
            class_probabilities_grouped_actual = class_probabilities_actual.reshape(-1, 3)
            confidence_grouped_actual = confidence_actual.reshape(-1,2)




            test_prob = yolo_resnet(xtest)
            test_prob = test_prob.cpu()



            test_prob_segments = [
                    test_prob[:segment_lengths[0]],
                    test_prob[segment_lengths[0]:segment_lengths[0] + segment_lengths[1]],
                    test_prob[segment_lengths[0] + segment_lengths[1]:]
                ]


            class_probabilities_predicted = test_prob_segments[0]  # Segment 1
            confidence_predicted = test_prob_segments[1]  # Segment 2
            bounding_boxes_predicted = test_prob_segments[2]  # Segment 3

            bounding_boxes_grouped_predicted = bounding_boxes_predicted.reshape(-1,2,4)
            class_probabilities_grouped_predicted = class_probabilities_predicted.reshape(-1, 3)
            confidence_grouped_predicted = confidence_predicted.reshape(-1,2)


            test_loss = custom_loss(test_prob, ytest)

            test_loss_for_mean.append(test_loss)


            iou_list = find_average_iou(bounding_boxes_grouped_actual, bounding_boxes_grouped_predicted)


            if len(iou_list) == 0:
                print("iou_list = 0")
                average_test_iou = 0
            else:
              average_test_iou = sum(iou_list)/len(iou_list)
              print(iou_list)
            test_iou_list.append(average_test_iou)


            # test_pred = torch.max(test_prob,1).indices
            # test_acc += int(torch.sum(test_pred == ytest))


    average_test_loss = sum(test_loss_for_mean)/len(test_loss_for_mean)
    test_losses.append(average_test_loss)

    # Calculate and pprint validation accuracy
    print(f"Epoch:- {epoch+1}, Train Loss:- {average_train_loss}, Test Loss:- {average_test_loss}, Train IOU:- {average_train_iou}, Test IOU:- {average_test_iou}")



    epc_train_df = pd.DataFrame({"Loss":[average_train_loss.detach().numpy()],"IOU":[average_train_iou.numpy()]})
    train_loss_csv = pd.concat([train_loss_csv, epc_train_df], ignore_index = True)
    train_loss_csv.to_csv("yolo_resnet_trn_loss_1.csv")

    epc_test_df = pd.DataFrame({"Loss":[average_test_loss.detach().numpy()],"IOU":[average_test_iou.numpy()]})

    test_loss_csv = pd.concat([test_loss_csv, epc_test_df], ignore_index = True)
    test_loss_csv.to_csv("yolo_resnet_test_loss_1.csv")

    # with open('yolo_model.pkl', 'wb') as file:
    #   # A new file will be created
    #   pickle.dump(yolo_resnet, file)



# print(type(), train_generator[1].shape)
# print(yolo_resnet(training_image))


2batch [00:00,  5.68batch/s]

[]
[]


4batch [00:00,  5.63batch/s]

[]
[]


6batch [00:01,  6.16batch/s]

...

[]
[]


136batch [00:20,  6.57batch/s]

[]
[]


136batch [00:20,  6.50batch/s]


KeyboardInterrupt: 